<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/fast_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os
import shutil
  
!git clone https://www.github.com/keras-team/keras-contrib.git \
  && cd keras-contrib \
  && pip install git+https://www.github.com/keras-team/keras-contrib.git \
  && python convert_to_tf_keras.py \
  && USE_TF_KERAS=1 python setup.py install

!pip install scipy==1.1.0

fatal: destination path 'keras-contrib' already exists and is not an empty directory.


In [3]:
import scipy
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Embedding, Lambda
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam
from keras.models import load_model, save_model

import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import numpy as np
import os
from glob import glob
from PIL import Image

import tensorflow as tf
from tensorflow.python.keras.backend import set_session, clear_session
tf.compat.v1.disable_v2_behavior()

from tqdm import tqdm

import cv2

from keras.applications.vgg19 import preprocess_input, VGG19

from keras.losses import mean_squared_error
from keras import backend as k

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
# source: https://www.machinecurve.com/index.php/2020/02/10/using-constant-padding-reflection-padding-and-replication-padding-with-keras/
from keras.layers import Layer

'''
  2D Reflection Padding
  Attributes:
    - padding: (padding_width, padding_height) tuple
'''
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

In [5]:
def conv_layer(x, num_filters, kernel_size, stride):
  reflection_size = kernel_size // 2
  reflection_padding = (reflection_size, reflection_size)
  x = ReflectionPadding2D(reflection_padding)(x)
  x = Conv2D(filters=num_filters, kernel_size=kernel_size, strides=stride)(x)
  return x

In [6]:
def residual_block(x, num_filters):
  input = x
  x = conv_layer(x, num_filters, 3, 1)
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)

  x = conv_layer(x, num_filters, 3, 1)
  x = InstanceNormalization()(x)

  x = Concatenate(axis=-1)([input, x])
  return x

In [7]:
def deconv_layer(x, num_filters, kernel_size, stride, upsample_size):
  reflection_size = kernel_size // 2
  reflection_padding = (reflection_size, reflection_size)
  x = UpSampling2D(size=upsample_size)(x)
  x = ReflectionPadding2D(reflection_padding)(x)
  x = Conv2D(filters=num_filters, kernel_size=kernel_size, strides=stride)(x)
  return x

In [42]:
def style_net(input):
  x = conv_layer(input, 32, 9, 1) #conv1
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 64, 3, 2) # conv2
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 128, 3, 2) # conv3
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = residual_block(x, 128) # res1
  x = residual_block(x, 128) # res2
  x = residual_block(x, 128) # res3
  x = residual_block(x, 128) # res4
  x = residual_block(x, 128) # res5
  x = deconv_layer(x, 64, 3, 1, 2) # deconv1
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = deconv_layer(x, 32, 3, 1, 2) # deconv2
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)
  x = conv_layer(x, 3, 9, 1) # deconv3? apparently.

  model = Model(input=input, output=x)
  return model

def style_model(net):
  input_img = Input(shape=img_shape)
  gen_img = net(input_img)
  # lambda function to normalize
  normalized_gen_img = Lambda(normalize_imgs)(gen_img)
  gen_features = extractor(normalized_gen_img)  
  gen_style_features = gen_features[:num_style_layers]
  gen_gram_style_features = [Lambda(gram_matrix)(gen_style_feature) for gen_style_feature in gen_style_features]
  gen_content_features = gen_features[num_style_layers:]
  
  gen_features = gen_gram_style_features + gen_content_features

  model = Model(input=input_img, output=gen_features)

  # losses = [get_style_loss for i in range(num_style_layers)] + [get_content_loss for i in range(num_content_layers)]
  losses = ['mean_squared_error' for i in range(len(gen_features))]

  weight_per_style_layer = style_weight / num_style_layers
  weight_per_content_layer = content_weight / num_content_layers

  loss_weights = [ weight_per_style_layer  * pair[1] for i, pair in enumerate(style_weights.items()) ] + [ weight_per_content_layer for i in range(num_content_layers) ]

  model.compile(loss=losses, loss_weights=loss_weights, optimizer=Adam(learning_rate=0.00002))

  return model

In [9]:
def load_image(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (img_shape[0], img_shape[1]))
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.expand_dims(img, axis=0)
  return img


In [38]:
def feature_extractor(layer_names, model):
  outputs = [model.get_layer(name).output for name in layer_names]
  model = Model(inputs=[vgg.input], outputs=outputs)
  model.trainable = False
  return model

In [11]:
def preprocess_imgs(imgs):
  imgs = imgs * 255.
  imgs = preprocess_input(imgs)
  return imgs

def normalize_imgs(imgs, batch_size=1):
  mean = np.array([0.485, 0.456, 0.406]).reshape([batch_size, 1, 1, -1])
  std = np.array([0.229, 0.224, 0.225]).reshape([batch_size, 1, 1, -1])
  imgs = imgs / 255.
  return (imgs - mean) / std

In [28]:
def gram_matrix(tensor):
  temp = tensor
  batch_size, height, width, channels = temp.get_shape().as_list()
  fun = tf.reshape(temp, [channels, height * width])
  result = tf.matmul(fun, fun, transpose_a=True)
  gram = tf.expand_dims(result, axis=0)
  gram = gram / (1. * channels * height * width)
  return gram

In [13]:
# def get_style_loss(gen_style, target_style):
#   # gen_style = gram_matrix(gen_style)
#   return tf.keras.losses.MeanAbsoluteError()(target_style, gen_style)

# def get_content_loss(gen_content, target_content):
#   return tf.keras.losses.MeanAbsoluteError()(target_content, gen_content)

# def get_total_loss(gen_features, target_features):
#   target_style_features = target_features[:num_style_layers]
#   target_content_features = target_features[num_style_layers:]

#   gen_style_features = gen_features[:num_style_layers]  

#   total_style_loss = sum([get_style_loss(gen_feature, target_feature) for gen_feature, target_feature in zip(gen_style_features, target_style_features)])

#   gen_content_features = gen_features[num_style_layers:]
#   total_content_loss = sum([get_content_loss(gen_feature, target_feature) for gen_feature, target_feature in zip(gen_content_features, target_content_features)])

#   total_loss = style_weight * total_style_loss + content_weight * total_content_loss

#   return total_loss, total_style_loss, total_content_loss

In [14]:
# @tf.function()
# def train_step(input_img):
#   with tf.GradientTape() as tape: # auto calculates gradinets
#     outputs = model(input_img)
#     print(outputs)
#     loss, _, _ = get_total_loss(outputs, input_features)

#   grad = tape.gradient(loss, model.trainable_weights)

#   opt.apply_gradients([(grad, model.trainable_weights)])

#   image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)) # clip pixels to be in range of [0, 1]

In [43]:
# vgg + style net setup
vgg = VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1','block5_conv1']
style_weights = { 
    'block1_conv1': 1., 
    'block2_conv1': 0.8,
    'block3_conv1': 0.5,
    'block4_conv1': 0.3,
    'block5_conv1': 0.1
}
content_layers = ['block4_conv2']

num_style_layers = len(style_layers)
num_content_layers = len(content_layers)

extractor = feature_extractor(style_layers + content_layers, vgg)

opt = tf.optimizers.Adam(learning_rate=0.02)

# image setup
img_shape = (128, 128, 3)
project_path = '/content/drive/My Drive/hairy_gan/'
style_img_path = os.path.join(project_path, 'style.jpg')
input_img_path = os.path.join(project_path, 'content.jpg')
input_img = load_image(input_img_path)
style_img = load_image(style_img_path)

processed_style_img = preprocess_imgs(style_img)
normalize_style_img = normalize_imgs(processed_style_img)
style_features = extractor(normalize_style_img)[:num_style_layers]
gram_style_features = [gram_matrix(feature) for feature in style_features]

processed_input_img = preprocess_imgs(input_img) # x
normalize_input_img = normalize_imgs(processed_input_img)
input_content_features = extractor(normalize_input_img)[num_style_layers:] # features_x
target_img = processed_input_img

In [44]:
epochs = 100
steps_per_epoch = 1
content_weight = 10
style_weight = 100

input = Input(shape=img_shape)
net = style_net(input)
model = style_model(net)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`


In [ ]:
for epoch in range(epochs):
  model.fit(target_img, gram_style_features + input_content_features, steps_per_epoch=steps_per_epoch)

Epoch 1/1
1/1 [==============================] - 14s 14s/step - loss: 9134.1191 - lambda_27_loss: 7.0468e-09 - lambda_28_loss: 1.8164e-04 - lambda_29_loss: 0.0042 - lambda_30_loss: 14.6896 - lambda_31_loss: 0.0069 - model_18_loss: 904.5924
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 9113.2812 - lambda_27_loss: 7.1807e-09 - lambda_28_loss: 1.8187e-04 - lambda_29_loss: 0.0041 - lambda_30_loss: 14.6847 - lambda_31_loss: 0.0069 - model_18_loss: 902.5115
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 9091.6670 - lambda_27_loss: 7.2546e-09 - lambda_28_loss: 1.8199e-04 - lambda_29_loss: 0.0041 - lambda_30_loss: 14.6770 - lambda_31_loss: 0.0069 - model_18_loss: 900.3547
Epoch 1/1
1/1 [==============================] - 7s 7s/step - loss: 9054.8486 - lambda_27_loss: 7.2901e-09 - lambda_28_loss: 1.8201e-04 - lambda_29_loss: 0.0041 - lambda_30_loss: 14.6671 - lambda_31_loss: 0.0069 - model_18_loss: 896.6788
Epoch 1/1
1/1 [==============================]

In [37]:
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
model_13 (Model)                (None, 128, 128, 3)  3519265     input_12[0][0]                   
__________________________________________________________________________________________________
lambda_14 (Lambda)              (None, 128, 128, 3)  0           model_13[1][0]                   
__________________________________________________________________________________________________
model_12 (Model)                multiple             12944960    lambda_14[0][0]                  
___________________________________________________________________________________________